In [1]:
from datetime import datetime
from Bio.Seq import Seq
import csv
import dateparser
import math
import numpy as np
import pandas as pd
import re
import seaborn as sns
import string

In [2]:
DATE = "082721"

In [3]:
nextclade_data = pd.read_csv("nextclade.tsv", sep="\t")

b'Skipping line 770: expected 1480 fields, saw 1481\n'
/Users/justindu/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (13,18,101,131,205,227,301,305,312,400,402,421,442,446,468,472,519,541,542,547,574,582,608,611,616,620,722,730,761,784,827,841,866,899,906,911,913,924,955,958,962,984,998,1004,1007,1012,1015,1017,1052,1130,1133,1134,1161,1170,1173,1179,1197,1200,1224,1238,1289,1309,1326,1340,1346,1353,1359,1384,1389,1401,1408) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## Create Map of Locations

In [8]:
"""
c("nsp1", "nsp2", "nsp3", "nsp4", "3CL-PRO", "nsp6", "nsp7", "nsp8", "nsp9", "nsp10",
                "Pol", "Hel", "ExoN", "nsp15", "nsp16", "S glycoprotein", "ORF3a", "E", "M", "ORF6", "ORF7a", "ORF7b",
                "ORF8", "N")
"""

gene_locations = {
    "nsp1": {
        "start": 266,
        "end": 805,
    },
    "nsp2": {
        "start": 806,
        "end": 2719,
    },
    "nsp3": {
        "start": 2720,
        "end": 8554
    },
    "nsp4": {
        "start": 8555,
        "end": 10054
    },
    "3CL-PRO": {
        "start": 10055,
        "end": 10972
    },
    "nsp6": {
        "start": 10973,
        "end": 11842
    },
    "nsp7": {
        "start": 11843,
        "end": 12091
    },
    "nsp8": {
        "start": 12092,
        "end": 12685
    },
    "nsp9": {
        "start": 12686,
        "end": 13024
    },
    "nsp10": {
        "start": 13025,
        "end": 13441
    },
    "Pol": {
        "start": 13462,
        "end": 16236
    },
    "Hel": {
        "start": 16237,
        "end": 18039
    },
    "ExoN": {
        "start": 18040,
        "end": 19620
    },
    "nsp15": {
        "start": 19621,
        "end": 20658
    },
    "nsp16": {
        "start": 20659,
        "end": 21552
    },
    "S glycoprotein": {
        "start": 21599,
        "end": 25381
    },
    "ORF3a": {
        "start": 25393,
        "end": 26217
    },
    "E": {
        "start": 26245,
        "end": 26469
    },
    "M": {
        "start": 26523,
        "end": 27188
    },
    "ORF6": {
        "start": 27202,
        "end": 27384
    },
    "ORF7a": {
        "start": 27439,
        "end": 27756
    },
    "ORF8": {
        "start": 27939,
        "end": 28256
    },
    "N": {
        "start": 28274,
        "end": 29530
    },
}

REFERENCE_SEQUENCE_NT = "..."
NUCLEOTIDES = ["A", "T", "G", "C"]

## Create Table of Gene, Location, Reference, Alternate

### Get a unique set of all mutations

In [9]:
nt_substitutions = nextclade_data["substitutions"].tolist()
unique_nt_substitutions = {}

In [10]:
def get_gene(mutation):
    location = int(mutation[1:-1])
    for gene in gene_locations:
        if location >= gene_locations[gene]["start"] and location <= gene_locations[gene]["end"]:
            return gene
    return None

# for mutation in unique_nt_substitutions:
# Should be nsp16
# mutation = "A20666T"
# print(get_gene(mutation))

In [11]:
def get_gene_sequence(gene):
    # Subtract by one because we start from 0 index, no + 1 at the end because same reason
    # For example, T490C
    # We want 489 since 489 - 0 + 1 = 490
    return REFERENCE_SEQUENCE_NT[gene_locations[gene]["start"] - 1 : gene_locations[gene]["end"]]

In [12]:
def get_nt_change_info(mutation):
    return mutation[0], int(mutation[1:-1]), mutation[-1]

In [13]:
for idx, substitution in enumerate(nt_substitutions):
    if isinstance(substitution, str):
        # Get both mutation lists
        current_nt_substitutions = substitution.split(",")
        # Go through all the amino acid substitutions
        for current_nt_substitution in current_nt_substitutions:
            if current_nt_substitution not in unique_nt_substitutions:
                unique_nt_substitutions[current_nt_substitution] = {"mutation": current_nt_substitution,
                                                                    "count": 1,
                                                                    "gene": get_gene(current_nt_substitution)}
            else:
                unique_nt_substitutions[current_nt_substitution]["count"] += 1

In [14]:
for nt_substitution in unique_nt_substitutions:
    # A gene we care about
    gene = unique_nt_substitutions[nt_substitution]["gene"]
    # Make sure it's not None
    if gene is not None:
        # Get the information about the mutation
        original_nt, location, new_nt = get_nt_change_info(nt_substitution)
        # Get the original sequence for the gene
        gene_sequence_original = Seq(get_gene_sequence(gene))
        # Adjust index since starting from 0 here
        # For example T490C, gene is nsp1, which has "start": 266 and "end": 805,
        # If we want the 490th position
        # We need to adjust since we chopped the gene
        # 490th is really 490 - 266 = 224 + 1 = 225th nt in the sequence, so we don't do the - 1
        gene_sequence_new = Seq(gene_sequence_original[:location - gene_locations[gene]["start"]] +
                                new_nt +
                                gene_sequence_original[location - gene_locations[gene]["start"] + 1:])
        # Translate both sequences
        aa_sequence_original = gene_sequence_original.translate()
        aa_sequence_new = gene_sequence_new.translate()
        # Now we want to see if it's synonomyous or not
        difference_indices = [i for i in range(len(aa_sequence_original)) if aa_sequence_original[i] != aa_sequence_new[i]]
        if len(difference_indices) > 1:
            print("Uh oh, more than two places different, shouldn't be the case")
        elif len(difference_indices) == 0:
            is_synonymous = True
            ## Amino Acid Location
            aa_location = math.ceil((location - gene_locations[gene]["start"] + 1) / 3) - 1
            wt_aa = aa_sequence_original[aa_location]
            new_aa = aa_sequence_original[aa_location]
            assert aa_sequence_original[aa_location] == aa_sequence_new[aa_location]
        elif len(difference_indices) == 1:
            # print(gene_locations[gene]["start"], gene_locations[gene]["end"], location, difference_indices)
            is_synonymous = False
            aa_location = difference_indices[0]
            wt_aa = aa_sequence_original[difference_indices[0]]
            new_aa = aa_sequence_new[difference_indices[0]]
        else:
            print("Nothing happened but something should have happened")
        unique_nt_substitutions[nt_substitution]["wt"] = wt_aa
        unique_nt_substitutions[nt_substitution]["alt"] = new_aa
        unique_nt_substitutions[nt_substitution]["synonymous"] = is_synonymous
        unique_nt_substitutions[nt_substitution]["aa index"] = aa_location
    else:
        unique_nt_substitutions[nt_substitution]["wt"] = None
        unique_nt_substitutions[nt_substitution]["alt"] = None
        unique_nt_substitutions[nt_substitution]["synonymous"] = None
        unique_nt_substitutions[nt_substitution]["aa index"] = None

## Write to a CSV

In [15]:
columns_of_interest = ["mutation", "gene", "count", "synonymous", "wt", "alt", "aa index"]
with(open(f'mutation_annotations_{DATE}.csv', 'w', newline='')) as file:
    fieldnames = columns_of_interest
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    
    writer.writeheader()
    
    for nt_substitution, metadata in unique_nt_substitutions.items():
        writer.writerow(metadata)